In [10]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
import shutil
import nibabel as nib
from subprocess import run

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [38]:
df=pd.DataFrame(columns=['class','data','label1','label2'])
count=0
class_list=['ADC','DWI']
for i in range(len(class_list)):
    data_list=glob(f'../../data/stroke_MRI_preprocessed/label/{class_list[i]}/**/')
    for j in tqdm(range(len(data_list))):
        label1=0
        label2=0
        file_list=glob(f'{data_list[j]}/*.png')
        file_list.sort()
        for k in range(len(file_list)):
            image=Image.open(file_list[k])
            image=np.array(image)
            if np.where(image==255)[0].shape[0]>0:
                if label1==0:
                    label1=k+1
                    label2=k+1
                else:
                    label2=k+1
        df.loc[count]=[class_list[i],os.path.basename(os.path.splitext(data_list[j])[0][:-1]),label1,label2]
        count+=1
df.to_csv('../../data/stroke_MRI_preprocessed/label.csv',index=False)

100%|██████████| 1325/1325 [01:28<00:00, 14.93it/s]


In [ ]:
df=pd.read_csv('../../data/stroke_MRI_preprocessed/label.csv')
flie_path='../../data/stroke_MRI_preprocessed/image/'
save_path='../../data/2D_MRI/'
for i in tqdm(range(len(df))):
    file_info=df.loc[i]
    class_path=file_info['class']
    data_path=file_info['data']
    label1=file_info['label1']
    label2=file_info['label2']
    label=int((label1+label2)/2)
    if label>20:
        create_dir(f'{save_path}{class_path}')
        shutil.copy(f'{flie_path}{class_path}/{data_path}/{str(label).zfill(3)}.png',f'{save_path}{class_path}/{data_path}.png')

100%|██████████| 2648/2648 [00:14<00:00, 186.31it/s]


In [28]:
input_folder = "../../data/stroke_MRI_preprocessed/image/ADC/ANO1_0078/"
output_nifti = "../../data/temp/input_mri.nii.gz"
output_brain_mask = "../../data/temp/brain_mask.nii.gz"
image_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".png")])
image_list = []
for filename in image_files:
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 흑백 이미지로 로드
    image_list.append(img)
image_stack = np.stack(image_list, axis=0)
nifti_image = nib.Nifti1Image(image_stack, affine=np.eye(4))
nib.save(nifti_image, output_nifti)
run(["hd-bet", "-i", output_nifti, "-o", output_brain_mask, "-device", "cuda"])
# 5️⃣ 마스크 파일 로드 후 뇌가 있는 슬라이스 찾기
brain_mask_nii = nib.load(output_brain_mask)
brain_mask_data = brain_mask_nii.get_fdata()  # Shape: (Z, H, W) -> (슬라이스 개수, 높이, 너비)

# 6️⃣ 뇌가 포함된 슬라이스 찾기
brain_slices = np.where(np.any(brain_mask_data > 0, axis=(1, 2)))[0]

if len(brain_slices) > 0:
    first_slice = brain_slices[0]  # 뇌가 처음 등장하는 슬라이스 번호
    last_slice = brain_slices[-1]  # 뇌가 마지막 등장하는 슬라이스 번호
    print(f"✅ 뇌가 포함된 슬라이스: {first_slice}번 ~ {last_slice}번")
else:
    print("⚠️ 뇌가 포함된 슬라이스가 없습니다!")


########################
If you are using hd-bet, please cite the following papers:

Isensee F, Schell M, Tursunova I, Brugnara G, Bonekamp D, Neuberger U, Wick A, Schlemmer HP, Heiland S, Wick W, Bendszus M, Maier-Hein KH, Kickingereder P. Automated brain extraction of multi-sequence MRI using artificial neural networks. arXiv preprint arXiv:1901.11341, 2019.

Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
########################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/LeeYS/bin/hd-bet", line 8, in <module>
    sys.exit(main())
  File "/home/ubuntu/anaconda3/envs/LeeYS/lib/python3.10/site-packages/HD_BET/entry_point.py", line 53, in main
    hdbet_predict(args.input,args.output, predictor, keep_brain_mask=args.save_bet_mask,
  File "/home/ubuntu/anaconda3/envs/LeeYS/lib/python3.10/site-packages/HD_BET/hd_bet_prediction.py", line 69, in hdbet_predict
    predictor.predict_from_files(
  File "/home/ubuntu/anaconda3/envs/LeeYS/lib/python3.10/site-packages/nnunetv2/inference/predict_from_raw_data.py", line 270, in predict_from_files
    return self.predict_from_data_iterator(data_iterator, save_probabilities, num_processes_segmentation_export)
  File "/home/ubuntu/anaconda3/envs/LeeYS/lib/python3.10/site-packages/nnunetv2/inference/predict_from_raw_data.py", line 363, in predict_from_data_iterator
    for preprocessed in data_iterator:
  File "/home/ubuntu/anaconda3/envs/LeeYS/lib/pyt

KeyboardInterrupt: 

In [17]:
image_stack.shape

(80, 256, 256)